<a href="https://colab.research.google.com/github/Vaieo/Initial/blob/main/mesh_grid_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%

# from utils import *
from matplotlib.patches import Rectangle, Polygon
import random
import numpy as np
import pandas as pd
import cv2 as cv
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import time
import pickle
#%%
img_name = "both_camera/right_camera\DSC_0110.JPG"
img = plt.imread(img_name)[:,:,0]/255
# --> threshold variable text box is required
threshold = 0.9
img[img<threshold] = 0
# ---> To do: Cropped image should be taken here
plt.imshow(img,cmap="gray")
plt.colorbar()
plt.show()
#%%
## Compute size of aperture
row_size = col_size = -np.inf
row_num = col_num = 0
for row in range(img.shape[0]):
    row_len = sum(img[row,:]>0)
    if row_len > row_size:
        row_size = row_len
        row_num = row
for col in range(img.shape[1]):
    col_len = sum(img[:,col]>0)
    if col_len > col_size:
        col_size = col_len
        col_num = col
print("Row size and col size",row_size, col_size)
# ---> To do: Give text boxes for taking this input
# aperture dia in mm
aperture_dia = 50
## spacing required(mm)
spacing = 1
## Create pixel spacing for mesh points
pixel_spacing_row = row_size*spacing//aperture_dia
pixel_spacing_column = col_size*spacing//aperture_dia

## Create mesh points
row_index = 0
mesh_points = []
while row_index < img.shape[0]:
    if np.any(img[row_index,:]>0):
        index = np.argmax(img[row_index,:]>0)
        while index < img.shape[1]:
            top_pixel = not ((row_index-pixel_spacing_column>0) and img[row_index-pixel_spacing_column,index]<=0)
            bottom_pixel =  not ((row_index+pixel_spacing_column<img.shape[0]) and img[row_index+pixel_spacing_column,index]<=0)
            left_pixel = not ((index-pixel_spacing_row>0) and img[row_index,index-pixel_spacing_row]<=0)
            right_pixel = not ((index+pixel_spacing_row<img.shape[1]) and img[row_index,index+pixel_spacing_row]<=0)
            if img[row_index,index]>0 and any([top_pixel,bottom_pixel,left_pixel,right_pixel]):

                mesh_points.append([row_index,index])
            index+=pixel_spacing_row
        row_index += pixel_spacing_column
    else:
        row_index+=1

img = plt.imread(img_name)/255
mesh_points = np.array(mesh_points)
# ---> To do: Substitute with file location and filename
filelocation = "31052024_Both_Camera_test/right_aperture"
filename = "right_aperture_mesh"
with open(f"{filelocation}/{filename}.pkl","wb") as f:
    pickle.dump(mesh_points,f)
plt.imshow(img,cmap="gray")
plt.scatter(mesh_points[:,1],mesh_points[:,0],s=5)
plt.colorbar()
plt.show()

# %%


In [ ]:
from tkinter import *


In [ ]:
import tkinter as tk

root = tk.Tk()
root.title("image processing gui")
tk.mainloop()

TclError: no display name and no $DISPLAY environment variable

In [ ]:
import tkinter as tk
from tkinter import filedialog
import matplotlib.pyplot as plt
import pickle
import os

class MeshGeneratorApp:
    def __init__(self, master):
        self.master = master
        master.title("Mesh Generator")

        self.label = tk.Label(master, text="Mesh Generator")
        self.label.pack()

        self.select_image_button = tk.Button(master, text="Select Image", command=self.select_image)
        self.select_image_button.pack()

        self.generate_mesh_button = tk.Button(master, text="Generate Mesh", command=self.generate_mesh)
        self.generate_mesh_button.pack()

        self.canvas = tk.Canvas(master, width=400, height=300)
        self.canvas.pack()

        self.mesh_points = []

    def select_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
        if file_path:
            self.img_name = file_path
            self.load_image()

    def load_image(self):
        img = plt.imread(self.img_name)[:,:,0]/255
        self.img_plot = plt.imshow(img, cmap="gray")
        plt.colorbar()
        plt.title("Loaded Image")
        plt.show()

    def generate_mesh(self):
        if hasattr(self, 'img_name'):
            # Mesh generation code
            img = plt.imread(self.img_name)[:,:,0]/255
            threshold = 0.9
            img[img<threshold] = 0
            row_size = col_size = -float('inf')
            row_num = col_num = 0
            for row in range(img.shape[0]):
                row_len = sum(img[row,:]>0)
                if row_len > row_size:
                    row_size = row_len
                    row_num = row
            for col in range(img.shape[1]):
                col_len = sum(img[:,col]>0)
                if col_len > col_size:
                    col_size = col_len
                    col_num = col
            aperture_dia = 50
            spacing = 1
            pixel_spacing_row = row_size*spacing//aperture_dia
            pixel_spacing_column = col_size*spacing//aperture_dia
            row_index = 0
            mesh_points = []
            while row_index < img.shape[0]:
                if np.any(img[row_index,:]>0):
                    index = np.argmax(img[row_index,:]>0)
                    while index < img.shape[1]:
                        top_pixel = not ((row_index-pixel_spacing_column>0) and img[row_index-pixel_spacing_column,index]<=0)
                        bottom_pixel =  not ((row_index+pixel_spacing_column<img.shape[0]) and img[row_index+pixel_spacing_column,index]<=0)
                        left_pixel = not ((index-pixel_spacing_row>0) and img[row_index,index-pixel_spacing_row]<=0)
                        right_pixel = not ((index+pixel_spacing_row<img.shape[1]) and img[row_index,index+pixel_spacing_row]<=0)
                        if img[row_index,index]>0 and any([top_pixel,bottom_pixel,left_pixel,right_pixel]):
                            mesh_points.append([row_index,index])
                        index+=pixel_spacing_row
                    row_index += pixel_spacing_column
                else:
                    row_index+=1
            self.mesh_points = mesh_points

            # Save mesh points to a file
            filelocation = os.path.dirname(self.img_name)
            filename = "mesh_points.pkl"
            with open(os.path.join(filelocation, filename), "wb") as f:
                pickle.dump(self.mesh_points, f)

            # Plot image with mesh points
            img = plt.imread(self.img_name)
            plt.imshow(img, cmap="gray")
            plt.scatter([point[1] for point in self.mesh_points], [point[0] for point in self.mesh_points], s=5)
            plt.colorbar()
            plt.title("Image with Mesh Points")
            plt.show()
        else:
            tk.messagebox.showerror("Error", "Please select an image first.")

root = tk.Tk()
app = MeshGeneratorApp(root)
root.mainloop()

TclError: no display name and no $DISPLAY environment variable

In [ ]:
import tkinter as tk
import customtkinter
from tkinter import filedialog, messagebox
import matplotlib.pyplot as plt
import pickle
import numpy as np
import os

class MeshGeneratorApp:
    def __init__(self, master):
        self.master = master
        master.title("Mesh Generator")

        screen_width = master.winfo_screenwidth()
        screen_height = master.winfo_screenheight()

        window_width = int(screen_width * 0.6)
        window_height = int(screen_height * 0.6)

        master.geometry(f"{window_width}x{window_height}")

        self.label = tk.Label(master, text="Mesh Generator")
        self.label.pack()
        # self.label.place(relx=0.5, rely=0.5, anchor=tk.CENTER)

        self.select_image_button = tk.Button(master, text="Select Image", command=self.select_image)
        self.select_image_button.pack()

        self.generate_mesh_button = tk.Button(master, text="Generate Mesh", command=self.generate_mesh)
        self.generate_mesh_button.pack()

        self.canvas = tk.Canvas(master, width=window_width, height=window_height)
        # self.canvas = tk.Canvas(master, width=screen_width * 0.5, height=screen_height * 0.5)
        # self.canvas = tk.Canvas(master, width=400, height=300)
        self.canvas.pack()

        self.mesh_points = []

    def select_image(self):
        script_dir = os.path.dirname(os.path.abspath(__file__))
        assets_dir = os.path.join(script_dir, 'assets')
        file_path = filedialog.askopenfilename(initialdir=assets_dir, filetypes=[("Image files", "*.jpg;*.png")])
        # initial_dir = os.path.dirname(os.path.abspath(__file__))
        # file_path = filedialog.askopenfilename(initialdir=initial_dir, filetypes=[("Image files", "*.jpg;*.png")])
        # file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.png")])
        if file_path:
            self.img_name = file_path
            self.load_image()

    def load_image(self):
        img = plt.imread(self.img_name)[:,:,0]/255
        self.img_plot = plt.imshow(img, cmap="gray")
        plt.colorbar()
        plt.title("Loaded Image")
        plt.show()

    def generate_mesh(self):
        if hasattr(self, 'img_name'):
            # Mesh generation code
            img = plt.imread(self.img_name)[:,:,0]/255
            threshold = 0.9
            img[img<threshold] = 0
            row_size = col_size = -float('inf')
            row_num = col_num = 0
            for row in range(img.shape[0]):
                row_len = sum(img[row,:]>0)
                if row_len > row_size:
                    row_size = row_len
                    row_num = row
            for col in range(img.shape[1]):
                col_len = sum(img[:,col]>0)
                if col_len > col_size:
                    col_size = col_len
                    col_num = col
            aperture_dia = 50
            spacing = 1
            pixel_spacing_row = row_size*spacing//aperture_dia
            pixel_spacing_column = col_size*spacing//aperture_dia
            row_index = 0
            mesh_points = []
            while row_index < img.shape[0]:
                if np.any(img[row_index,:]>0):
                    index = np.argmax(img[row_index,:]>0)
                    while index < img.shape[1]:
                        top_pixel = not ((row_index-pixel_spacing_column>0) and img[row_index-pixel_spacing_column,index]<=0)
                        bottom_pixel =  not ((row_index+pixel_spacing_column<img.shape[0]) and img[row_index+pixel_spacing_column,index]<=0)
                        left_pixel = not ((index-pixel_spacing_row>0) and img[row_index,index-pixel_spacing_row]<=0)
                        right_pixel = not ((index+pixel_spacing_row<img.shape[1]) and img[row_index,index+pixel_spacing_row]<=0)
                        if img[row_index,index]>0 and any([top_pixel,bottom_pixel,left_pixel,right_pixel]):
                            mesh_points.append([row_index,index])
                        index+=pixel_spacing_row
                    row_index += pixel_spacing_column
                else:
                    row_index+=1
            self.mesh_points = mesh_points

            # Save mesh points to a file
            filelocation = os.path.dirname(self.img_name)
            filename = "mesh_points.pkl"
            with open(os.path.join(filelocation, filename), "wb") as f:
                pickle.dump(self.mesh_points, f)

            # Plot image with mesh points
            img = plt.imread(self.img_name)
            plt.imshow(img, cmap="gray")
            plt.scatter([point[1] for point in self.mesh_points], [point[0] for point in self.mesh_points], s=5)
            plt.colorbar()
            plt.title("Image with Mesh Points")
            plt.show()
        else:
            tk.messagebox.showerror("Error", "Please select an image first.")

root = tk.Tk()
app = MeshGeneratorApp(root)
root.mainloop()